In [1]:
pip install speechbrain

In [2]:
!git clone https://huggingface.co/speechbrain/spkrec-ecapa-voxceleb

fatal: destination path 'spkrec-ecapa-voxceleb' already exists and is not an empty directory.


In [3]:
import torchaudio
import torch
import speechbrain as sb
from speechbrain.pretrained import EncoderClassifier
import sounddevice as sd
import numpy as np

# Load the Speaker Recognition Model
model = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="tmp/spkrec-ecapa-voxceleb"
)

# Function to Record Audio
def record_audio(filename, duration=5, sample_rate=16000):
    print(f"🎤 Recording {duration} seconds of audio...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype=np.float32)
    sd.wait()
    torchaudio.save(filename, torch.tensor(audio_data.T), sample_rate)
    print(f"✅ Recording saved as {filename}")

# Function to Denoise Audio (Placeholder)
def denoise_audio(waveform, sample_rate):
    return waveform  # Implement noise reduction here if needed

def enroll_speaker(audio_paths):
    embeddings = []
    for path in audio_paths:
        waveform, sample_rate = torchaudio.load(path)
        waveform = denoise_audio(waveform, sample_rate)  # Apply denoising

        # Ensure waveform has sufficient length
        if waveform.shape[-1] < 3:
            waveform = torch.nn.functional.pad(waveform, (0, 3 - waveform.shape[-1]))

        embedding = model.encode_batch(waveform, wav_lens=None)
        embeddings.append(embedding)

    return torch.stack(embeddings).mean(dim=0)

# Generate 10 Enrollment Audio Filenames
enrolled_audio_paths = [f"enroll_{i}.wav" for i in range(1, 11)]

# Duplicate the 10 Samples to Make 50
duplicated_audio_paths = enrolled_audio_paths * 5  # 10 samples × 5 = 50

# Record 10 Enrollment Samples
for path in enrolled_audio_paths:
    record_audio(path, duration=5)

# Generate Embedding for Enrollment (Now Using 50 Samples)
enrolled_voice = enroll_speaker(duplicated_audio_paths)


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
C:\Users\kuksh\anaconda3\lib\inspect.py:746: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
C:\Users\kuksh\AppData\Local\Temp\ipykernel_25564\2454809806.py:4: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from H

🎤 Recording 5 seconds of audio...


KeyboardInterrupt: 